### IMPORTS

In [1]:
import subprocess
import os
import json
import re

### CONSTANTS

In [2]:
BASE_DIR = "E:\Workspace\Work"

### HELPER FUNCTIONS

In [3]:
# remove all ANSI escape sequences from a string
def removeEscapeSequences(s):
    return re.sub(r'\x1b[^m]*m', '', s)

### UPDATE JSON FILES

In [18]:
# Update Projects
subprocess.run("glab api /projects?owned=yes > glab_projects.json",
               cwd=os.getcwd(), shell=True)
with open("glab_projects.json", "r") as f:
    projects = json.load(f)

# Update Groups
subprocess.run("glab api /groups?owned=yes > glab_groups.json",
               cwd=os.getcwd(), shell=True)
with open("glab_groups.json", "r") as f:
    groups = json.load(f)

# Update Repos
subprocess.run("gh api /user/repos > gh_repos.json",
               cwd=os.getcwd(), shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout.decode("utf-8")


with open("gh_repos.json", "r") as f:
    repos = json.loads(removeEscapeSequences(f.read()))

with open("gh_repos.json", "w") as f:
    json.dump(repos, f, indent=4)

# Filter out archived repos
repos = list(filter(lambda x: not x["archived"], repos))

### CREATE FOLDER STRUCTURE

In [5]:
for group in groups:
    # create group folder with full path
    group_folder = os.path.join(BASE_DIR, group["full_path"])
    os.makedirs(group_folder, exist_ok=True)

### CLONE GLAB PROJECTS INTO FOLDER STRUCTURE

In [6]:
for project in projects:
    # create project folder with full path
    project_folder = os.path.join(BASE_DIR, project["path_with_namespace"])

    # check if the project is already cloned
    if os.path.exists(os.path.join(BASE_DIR, project["path_with_namespace"])):
        print(f"Project {project['name']} is already cloned, pulling latest changes...")
        # TODO: implement pull command
        # # run glab pull command
        # subprocess.run(
        #     f"cd {project_folder} && glab pull",
        #     shell=True,
        #     check=True,
        # )
    else:
        # run glab clone command
        subprocess.run(
            f"glab repo clone {project['path_with_namespace']} {project_folder}",
            shell=True,
            check=True,
        )
        print(f"Cloned project {project['name']} to {project_folder}")

Project RealEstateScrapeBot is already cloned, pulling latest changes...
Project OneTrueGit is already cloned, pulling latest changes...
Project MAT120 is already cloned, pulling latest changes...
Project CSE421 is already cloned, pulling latest changes...
Project CSE250 is already cloned, pulling latest changes...
Project CSE221 is already cloned, pulling latest changes...
Project CSE220 is already cloned, pulling latest changes...
Project CSE111 is already cloned, pulling latest changes...
Project CSE110 is already cloned, pulling latest changes...
Project The Library is already cloned, pulling latest changes...
Project jupyter-ipyreact is already cloned, pulling latest changes...
Project Data Structure is already cloned, pulling latest changes...
Project LAB-temp is already cloned, pulling latest changes...
Project BracU-App is already cloned, pulling latest changes...
Project BookStore is already cloned, pulling latest changes...
Project CarRepairDashboardApp is already cloned, pul

### GOTO EACH PROJECT AND CREATE A GITHUB REPO

#### ADD THE GITHUB REPO AS REMOTE
#### PUSH THE PROJECT TO GITHUB

In [20]:
allPathWithNamespace = map(lambda x: x['path_with_namespace'], projects)


def gHubNamingFormat(q: str): return q.replace("/", "__")


def gLabNamingFormat(q: str): return q.replace("__", "/")



# TODO: THIS DOESNT ACCOUNT FOR THE VISIBILITY OF THE REPO. FUTURE PLAN: USE OOP OR DATABASE OR DATACLASS TO MANAGE THE ENTIRE APPLICATION
def syncWithGitHub(localPath: str):
    # get full path of the project
    repoPath = os.path.join(BASE_DIR, localPath)
    # generate the github repo name
    githubRepoName = gHubNamingFormat(localPath)
    # check if the project is already exists in github
    # TODO: implement check if the project is already exists in github
    # if not, create the repo
    os.chdir(repoPath)
    subprocess.run(
        f"gh repo create {githubRepoName} --public",
        shell=True,
        check=True,
    )
    repoUrl = f"https://github.com/SafwanAmin-BracU/{githubRepoName}.git"
    # add the github repo as remote to the project
    subprocess.run(
        f"git remote set-url --add --push origin {repoUrl}",
        shell=True,
        check=True,
    )


for pathWithNamespace in allPathWithNamespace:
    syncWithGitHub(pathWithNamespace)
    print(f"Synced project {pathWithNamespace} with GitHub")

Synced project breaktimecode/portfolio/webscraping/RealEstateScrapeBot with GitHub
Synced project community3940225/onetruegit with GitHub
Synced project bracu_23101281/lab/MAT120 with GitHub
Synced project bracu_23101281/lab/CSE421 with GitHub
Synced project bracu_23101281/lab/CSE250 with GitHub
Synced project bracu_23101281/lab/CSE221 with GitHub
Synced project bracu_23101281/lab/CSE220 with GitHub
Synced project bracu_23101281/lab/CSE111 with GitHub
Synced project bracu_23101281/lab/CSE110 with GitHub
Synced project breaktimecode/reaserch-and-development/the-library with GitHub
Synced project playground_23101281/jupyter-ipyreact with GitHub
Synced project learning_23101281/data-structure-and-algorithms/data-structure with GitHub
Synced project bracu_23101281/LAB-temp with GitHub
Synced project breaktimecode/bracu-projects/bracu-app with GitHub
Synced project bracu_23101281/group-projects/CSE111_SUM23_GroupProject with GitHub
Synced project breaktimecode/portfolio/design/carrepairdash